In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import pymannkendall as mk
import warnings
import logging

# ========== 关闭警告和日志 ==========
warnings.filterwarnings("ignore")
logging.getLogger('fiona').setLevel(logging.ERROR)
logging.getLogger('shapely').setLevel(logging.ERROR)
logging.getLogger('geopandas').setLevel(logging.ERROR)

# ========== 路径设置 ==========
shp_path = r"E:\00DIC全球\作图\导出图\正图1\DIC\SHP\lake_DIC.shp"
output_path = r"E:\00DIC全球\作图\导出图\正图1\DIC\20250618\lake_dic_mk_result.shp"

# ========== 读取文件 ==========
gdf = gpd.read_file(shp_path)

# ========== 提取DIC字段 ==========
year_cols = [f"DIC_{year}" for year in range(1984, 2025)]

# ========== 存储结果 ==========
slopes = []
p_values = []
valid_flags = []

# ========== 遍历每个湖泊 ==========
for idx, row in enumerate(gdf.itertuples(), 1):
    # 获取每年的DIC浓度，0当作无效值
    series = pd.Series([getattr(row, col) for col in year_cols]).replace(0, np.nan)
    valid_series = series.dropna()

    # 观测少于10个，跳过
    if valid_series.count() < 10:
        slopes.append(np.nan)
        p_values.append(np.nan)
        valid_flags.append(False)
        continue

    # 异常值剔除（IQR法）
    q1 = valid_series.quantile(0.25)
    q3 = valid_series.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    filtered = valid_series[(valid_series >= lower) & (valid_series <= upper)]

    # 剔除异常值后仍需至少10年数据
    if filtered.count() < 10:
        slopes.append(np.nan)
        p_values.append(np.nan)
        valid_flags.append(False)
        continue

    # Mann-Kendall 趋势检验
    try:
        result = mk.original_test(filtered.values)
        slopes.append(result.slope)
        p_values.append(result.p)
        valid_flags.append(True)
    except:
        slopes.append(np.nan)
        p_values.append(np.nan)
        valid_flags.append(False)

    # 每500个湖泊打印一次进度
    if idx % 500 == 0:
        print(f"已处理 {idx} / {len(gdf)} 个湖泊")

# ========== 写入结果 ==========
gdf["MK_slope"] = slopes
gdf["MK_p"] = p_values
gdf["MK_valid"] = valid_flags

# ========== 筛选有效结果 ==========
gdf_valid = gdf[gdf["MK_valid"]]

# ========== 保存为新的shapefile ==========
gdf_valid.to_file(output_path)

print(f"处理完成！共有效湖泊数量：{len(gdf_valid)}")
